# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.234226578278                   -0.50    8.0
  2   -7.250227580227       -1.80       -1.40    1.0
  3   -7.251316822968       -2.96       -2.16    4.0
  4   -7.251295869454   +   -4.68       -2.29    3.0
  5   -7.251330909420       -4.46       -2.72    2.0
  6   -7.251337231337       -5.20       -3.04    2.0
  7   -7.251338767613       -5.81       -3.94    1.0
  8   -7.251338794221       -7.58       -4.21    4.0
  9   -7.251338798236       -8.40       -4.73    2.0
 10   -7.251338798681       -9.35       -5.39    2.0
 11   -7.251338798699      -10.74       -5.64    3.0
 12   -7.251338798704      -11.35       -6.06    2.0
 13   -7.251338798705      -12.22       -6.86    3.0
 14   -7.251338798705      -14.01       -7.28    2.0


Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.0761188165435452
[ Info: Arnoldi iteration step 2: normres = 0.6176260667356475
[ Info: Arnoldi iteration step 3: normres = 0.7030464419834104
[ Info: Arnoldi iteration step 4: normres = 0.22770287344324094
[ Info: Arnoldi iteration step 5: normres = 0.24183650522483577
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (6.41e-03, 2.66e-02, 1.69e-01, 1.61e-01, 5.61e-02)
[ Info: Arnoldi iteration step 6: normres = 0.6025913971811977
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (2.85e-03, 5.49e-02, 5.78e-01, 1.25e-01, 4.24e-02)
[ Info: Arnoldi iteration step 7: normres = 0.10397430451184511
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (1.97e-04, 5.81e-02, 1.01e-02, 4.55e-02, 6.71e-02)
[ Info: Arnoldi iteration step 8: normres = 0.12158092013501101
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (1.03e-